In [274]:
import csv

stops = {}
stopsOnTrip = {}
routeStations = {}
routeTrips = {}
tripRoute = {}
routeShortNames = {}

with open('../gtfs_static_data_ul/stops.csv') as file:
    reader = csv.DictReader(file)
    # Create dictionary key for each row in stops.csv with values on name, latitude and longitude
    for row in reader:
        stops[row['stop_id']] = {'stop_id': row['stop_id'], 'stop_name': row['stop_name'],
                                 'stop_lat': row['stop_lat'], 'stop_lon': row['stop_lon']}

with open('../gtfs_static_data_ul/stop_times.csv') as file:
    reader1 = csv.DictReader(file)
    # Loops through stop_times.csv and creates dictionary with stops per unique trip
    for row in reader1:
        stopsOnTrip.setdefault(row['trip_id'],[]).append(stops[row['stop_id']])
        
with open('../gtfs_static_data_ul/trips.csv') as file:
    reader3 = csv.DictReader(file)
    # Creates a dictionary with all unique trip_id:s per unique route_id
    for row in reader3:
        routeTrips.setdefault((row['direction_id'],row['route_id']),[]).append(row['trip_id'])
        
with open('../gtfs_static_data_ul/routes.csv') as file:
    reader4 = csv.DictReader(file)
    # Creates a dictionary with all unique route_id:s and corresponding route_short_name
    for row in reader4:
        routeShortNames.setdefault(row['route_id'],[]).append(row['route_short_name'])
        
with open('../gtfs_static_data_ul/trips.csv') as file:
    reader5 = csv.DictReader(file)
    # Creates a dictionary with all unique trip_id:s per unique route_id
    for row in reader5:
        tripRoute[row['trip_id']] = { 'route':row['route_id'],'direction':row['direction_id'] }
        
    # Creates dictionary with all station data per route
for route in routeTrips:
    routeStations[route] = { 'stops': stopsOnTrip[routeTrips[route][0]] }

In [288]:
from google.transit import gtfs_realtime_pb2
import gzip
import os
import datetime

directory = '../data/30-small/'
datapoints = np.zeros((0,3), int)

# Loop through all files in directory
for filename in os.listdir(directory):
        
    # Uncompress and parse protobuff-file using gtfs_realtime_pb2
    with gzip.open(directory + filename, 'rb') as file:
        response = file.read()
        feed = gtfs_realtime_pb2.FeedMessage()
        feed.ParseFromString(response)
           
        print("Filename: " + filename, "Trips: " + str(len(feed.entity)))

        for trip in feed.entity:
            if (len(trip.trip_update.stop_time_update)-1==len(stopsOnTrip[trip.trip_update.trip.trip_id])):
                print(trip.trip_update.trip.trip_id, len(trip.trip_update.stop_time_update),"av",len(stopsOnTrip[trip.trip_update.trip.trip_id]))
                for i, stop_time_update in enumerate(trip.trip_update.stop_time_update):

                    # Excluding the last stop as it does not have a "departuring edge"
                    if (i<len(trip.trip_update.stop_time_update)-1):

                        # Store contextual metadata for the unique delay data point
                        ts = int(trip.trip_update.stop_time_update[i+1].arrival.time)
                        key = int(str(trip.trip_update.stop_time_update[i].stop_id) + str(trip.trip_update.stop_time_update[i+1].stop_id))

                        # Store the delay data point
                        delay = int(trip.trip_update.stop_time_update[i+1].arrival.delay-trip.trip_update.stop_time_update[i].arrival.delay)
                        datapoints = np.append(datapoints, np.array([[key,ts,delay]]), axis=0)

Filename: 1580424781404 Trips: 91


KeyError: '33010000129962895'

In [284]:
import pandas as pd

ID_col = datapoints[:,0]
TS_col = pd.to_numeric(datapoints[:,1])
Delay_col = pd.to_numeric(datapoints[:,2])

d = {'Edge': ID_col, 'Timestamp': TS_col, 'Delay': Delay_col}

df = pd.DataFrame(data=d)
df = pd.DataFrame.drop_duplicates(df)
df

,Edge,Timestamp,Delay


In [231]:
df.loc[df['Edge'] == 90220037003300069022003700552004]

,Edge,Timestamp,Delay
1447,90220037003300069022003700552004,1580425107,1
6755,90220037003300069022003700552004,1580425139,-15
16982,90220037003300069022003700552004,1580425031,2


In [237]:
arr = np.zeros((10,3),int)
arr

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])